# Building a Spam Filter with Naive Bayes

Using Multinomial Naive Bayes Theorem, we utilize the assumption of conditional independence between words within an SMS message and messages already classified as spam/non-spam to decide if a new message belongs to one category or another.

We have also learned to use Laplace Smoothing to correct for words that are present in the data set but not in the specific segment we are looking into (spam/non-spam).

The posterior distribution in a multinomial Bayesian setting is given by:

$$ P(\theta | X) \propto P(X | \theta) P(\theta) $$

where:
- theta represents the parameters (e.g., probabilities of categories in a multinomial distribution).
- \( X \) represents the observed data.
- P(X | theta) is the likelihood, which in the case of a multinomial distribution is:

$$ P(X | \theta) = \frac{N!}{x_1! x_2! \dots x_k!} \theta_1^{x_1} \theta_2^{x_2} \dots \theta_k^{x_k} $$

We are now going to use this theory and apply it to a practical example, where we will analyze a [real data set](https://archive.ics.uci.edu/ml/datasets/sms+spam+collection) put together by Tiago A. Almeida and José María Gómez Hidalgo from UC Irvine. We'll use the multinomial Naive Bayes algorithm along with the dataset's 5,572 SMS messages that are already classified by humans.

In [1]:
# import relevant libraries
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re
% matplotlib inline
plt.style.use('fivethirtyeight')

# read the csv file and use tabs as separators
sms = pd.read_csv('SMSSpamCollection', 
                  sep ='\t', 
                  header = None, 
                  names = ['Label', 'SMS'])

In [2]:
sms.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 2 columns):
Label    5572 non-null object
SMS      5572 non-null object
dtypes: object(2)
memory usage: 87.1+ KB


In [3]:
sms.describe()

,Label,SMS
count,5572,5572
unique,2,5169
top,ham,"Sorry, I'll call later"
freq,4825,30


Under the 'Label' column we have two distinct values: 
- spam: messages classified as spam
- ham: messages classified as non-spam

In [4]:
sms['Label'].value_counts(normalize = True)*100

ham     86.593683
spam    13.406317
Name: Label, dtype: float64

In [5]:
dup = sms[sms['SMS'].duplicated()].copy()

In [6]:
dup.describe()

,Label,SMS
count,403,403
unique,2,281
top,ham,"Sorry, I'll call later"
freq,309,29


In [7]:
dup['Label'].value_counts(normalize = True)*100

ham     76.674938
spam    23.325062
Name: Label, dtype: float64

So far, we know that we have:
- 5572 messages, of which 5169 have a distinct value
    - 76.7% of those duplicated messages are non-spam
- 86.6% of those are non-spam, 13.4% are spam

## Training and Test Set

Before coming up with the algorithm, we need to make sure that we will be able to properly test it. For that purpose, once our filter is done we are going to split our dataset in two:

- A training set to 'train' the computer on message classification
- A test set, which we will use to compare our algorithm's output to the spam label that already exists

To make sure that we have enough data for both tasks, we will use 80% of messages for training (4,458 messages), and the other 20% for testing (1,114 messages). 

Our goal for this project is to obtain a spam filter that classifies new messages as spam/non-spam. Therefore, if we can show that the label we will create for the 1,114 'new' messages matches 80%+ of the pre-existing, human-created labels, then this project will be considered a success.

We will first start by randomizing our entire dataset, using random state = 1 to make sure our results can be reproduced:

In [8]:
# randomize entire dataset
sms_randomized = sms.sample(frac=1, random_state=1).reset_index(drop = True)

# split dataset into training and test sets
train = sms_randomized.loc[0:4458, :].reset_index(drop = True).copy()
test = sms_randomized.loc[4458:, :].reset_index(drop = True).copy()

Before continuing, we want to make sure that the spam to non-spam ration is similar in both sets:

In [9]:
print(train['Label'].value_counts(normalize=True)*100, 
      '\n',
      test['Label'].value_counts(normalize=True)*100)

ham     86.544068
spam    13.455932
Name: Label, dtype: float64 
 ham     86.804309
spam    13.195691
Name: Label, dtype: float64


Ratios are very similar, so we can continue with the project.

## Letter Case and Punctuation

With out dataset split, we now have to use our training set to teach the algorithm to classify new messages.

To do so, we will continue to use the Multinomial Naive Algorithm so that:

$$
P(Spam | w_1,w_2,...,w_n)\ \alpha\ P(Spam) * \prod_{i=1}^n P(w_i | Spam)
$$
$$
P(Ham | w_1,w_2,...,w_n)\ \alpha\ P(Ham) * \prod_{i=1}^n P(w_i | Ham)
$$

Where:

$$
P(w_i|Spam) = \frac{N_{w_i | Spam} + \alpha}{N_{Spam} + \alpha * N_{Vocabulary}}
$$
$$
P(w_i|Ham) = \frac{N_{w_i | Ham} + \alpha}{N_{Ham} + \alpha * N_{Vocabulary}}
$$

In the equations above we have various terms:

$$ N_{w_i | Spam} =  \mbox{the  number  of  times  the  word w_i occurs in spam messages} $$
$$ N_{w_i | Spam^C} =  \mbox{the  number  of  times  the  word w_i occurs in non-spam messages} $$

$$ N_{w_i | Spam} =  \mbox{total number of words in spam messages} $$
$$ N_{w_i | Spam^C} =  \mbox{total number of words in non-spam messages} $$

$$ N_{Vocabulary} =  \mbox{total number of words in the vocabulary} $$
$$ \alpha =  \mbox{1 (alpha is a smoothing parameter)} $$


Before designing the algorithm, we need to do some data cleaning. To have messages that we can properly analyze, we need to remove everything that is not a character from a-z, A-Z or 0-9, and we will also want to convert everything to lower-case.

In [10]:
train.loc[:,'SMS'] = train['SMS'].apply(lambda x: re.sub('\W', ' ', x))
train.loc[:,'SMS'] = train['SMS'].str.lower()

## Creating the Vocabulary

We call the set of each individual words in our SMS column our 'vocabulary'. We need this set because eventually we will have an individual column for each unique word, and we need to create a list with these before that is possible.

In [11]:
vocab = [] # list with individual words

# create new column with messages split in a list
train['SMS_SPLIT'] = train['SMS'].apply(lambda x: x.split())
# iterate through list of words and append new words to vocabulary list
train['SMS_SPLIT'].apply(lambda x: [vocab.append(i) for i in x if i not in vocab])

# transform vocab list into a set to remove duplicates and then back into a list
vocab = set(vocab)
vocab = list(vocab)

## Finalizing Training Set

Now that we have our vocabulary list, we need to create a dictionary with the frequency count of each individual word. This will then be used to create the final version of our training dataset.

In [12]:
# initialize dictionary with 0s in all rows, with as many rows as messages in the training dataset
# Each column will be an individual word from the vocabulary
word_counts_per_sms = {key : [0] * len(train['SMS_SPLIT']) for key in vocab}

# loop through messages while using enumerate to get both index and the SMS message
for index, sms in enumerate(train['SMS_SPLIT']):
    for word in sms:
        word_counts_per_sms[word][index] += 1

With the code above, we should now have a dictionary where each key is a distinct word from the SMS messages. Each of those keys should have a list as long as the dataset as value. Within said list, the index corresponds to a message in the dataset with the same index, and the value corresponds to the number of times that the key appears in said message. 

Now the last step to get the training set ready is converting this dictionary to a data frame, and concatenating this data frame to our original training dataset:

In [13]:
new_cols = pd.DataFrame(word_counts_per_sms)
clean_train = pd.concat([train,new_cols], axis = 1)
clean_train.head()

,Label,SMS,SMS_SPLIT,0,00,000,000pes,008704050406,0089,01223585334,...,zindgi,zoe,zogtorius,zouk,zyada,é,ú1,ü,〨ud,鈥
0,ham,yep by the pretty sculpture,"[yep, by, the, pretty, sculpture]",0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,ham,yes princess are you going to make me moan,"[yes, princess, are, you, going, to, make, me,...",0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,ham,welp apparently he retired,"[welp, apparently, he, retired]",0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,ham,havent,[havent],0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,ham,i forgot 2 ask ü all smth there s a card on ...,"[i, forgot, 2, ask, ü, all, smth, there, s, a,...",0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,2,0,0


## Calculating Constants First

Now that we have the clean training data, we should recall that the formulas for the Naive Bayesian model are such:

$$
P(Spam | w_1,w_2,...,w_n)\ \alpha\ P(Spam) * \prod_{i=1}^n P(w_i | Spam)
$$
$$
P(Ham | w_1,w_2,...,w_n)\ \alpha\ P(Ham) * \prod_{i=1}^n P(w_i | Ham)
$$

Where:

$$
P(w_i|Spam) = \frac{N_{w_i | Spam} + \alpha}{N_{Spam} + \alpha * N_{Vocabulary}}
$$
$$
P(w_i|Ham) = \frac{N_{w_i | Ham} + \alpha}{N_{Ham} + \alpha * N_{Vocabulary}}
$$

Within those, some values never change. We can calculate the values for:

- P(Spam)
- P(Ham)
- $ N_{Spam}, N_{Ham}, N_{Vocabulary} $

Refer to the 'Letter Case and Punctuation' section for the meaning of each of these constants. Also, for this exercise we will use a Laplace constant of $ \alpha = 1$

In [14]:
# initiating all constants

spam = clean_train[clean_train['Label'] == 'spam']
ham = clean_train[clean_train['Label'] == 'ham']

p_spam = len(spam) / len(clean_train) # probability of message being spam
p_ham = len(ham) / len(clean_train) # probability of message being non-spam

# create series with number of words, then sum for total words per category
n_spam = spam['SMS_SPLIT'].apply(len).sum() # words in spam
n_ham = ham['SMS_SPLIT'].apply(len).sum() # words in ham

n_vocabulary = len(vocab) # words in vocabulary

alpha = 1

## Calculating Parameters

The constants above are 'constant' because their values do not change regardless of what word we are looking into. However, $P(w_i|Spam)$ and $P(w_i|Ham)$ do change, since the probability of each word changes according to whether the message is spam or not.

We will need to calculate:
$$
P(w_i|Spam) = \frac{N_{w_i | Spam} + \alpha}{N_{Spam} + \alpha * N_{Vocabulary}}
$$
$$
P(w_i|Ham) = \frac{N_{w_i | Ham} + \alpha}{N_{Ham} + \alpha * N_{Vocabulary}}
$$

The steps we take to calculate P("secret"|Spam) will be identical for both of our new messages above, or for any other new message that contains the word "secret". The key detail here is that calculating P("secret"|Spam) only depends on the training set, and as long as we don't make changes to the training set, P("secret"|Spam) stays constant. The same reasoning also applies to P("secret"|Ham).

This means that we can use our training set to calculate the probability for each word in our vocabulary. If our vocabulary contained only the words "lost", "navigate", and "sea", then we'd need to calculate six probabilities:

- P("lost"|Spam) and P("lost"|Ham)
- P("navigate"|Spam) and P("navigate"|Ham)
- P("sea"|Spam) and P("sea"|Ham)

We have 7,783 words in our vocabulary, which means we'll need to calculate a total of 15,566 probabilities. For each word, we need to calculate both $P(w_i|Spam)$ and $P(w_i|Ham)$.

In more technical language, the probability values that P(wi|Spam) and P(wi|Ham) will take are called __parameters__. 

The fact that we calculate so many values before even beginning the classification of new messages makes the Naive Bayes algorithm very fast (especially compared to other algorithms). When a new message comes in, most of the needed computations are already done, which enables the algorithm to almost instantly classify the new message.

Recall that P(wi|Spam) and P(wi|Ham) are key parts of the equations that we need to classify the new messages:

$$
P(Spam | w_1,w_2,...,w_n)\ \alpha\ P(Spam) * \prod_{i=1}^n P(w_i | Spam)
$$
$$
P(Ham | w_1,w_2,...,w_n)\ \alpha\ P(Ham) * \prod_{i=1}^n P(w_i | Ham)
$$

To calculate all of these we will initialize two dictionaries, one for spam and one for ham, where the keys are the individual words and their values are the the probabilities of said word appearing within their classification. Recall that we already formed a column for each word, so we just need to sum each column to check how many times said word appears in both spam and ham.

In [15]:
# creating dictionaries with SPAM and HAM parameters for entire vocabulary
spam_parameters = {distinct_word : 0 for distinct_word in vocab}
ham_parameters = {distinct_word : 0 for distinct_word in vocab}

# we have dataframes with SPAM and HAM, need probability of finding each word in each of those dataframes
for distinct_word in vocab:
    
    # number of times word happens in spam and ham will be the sum of the word's respective column
    n_word_spam = spam[distinct_word].sum() # times word appears in spam
    n_word_ham = ham[distinct_word].sum() # times word appears in non-spam
    
    # use these along with constants to get the probability of a word appearing in either group
    prob_word_given_spam = (n_word_spam + alpha) / (n_spam + alpha * n_vocabulary)
    prob_word_given_ham =  (n_word_ham + alpha) / (n_ham + alpha * n_vocabulary)
    
    # update parameters to their respective dictionary
    spam_parameters[distinct_word] = prob_word_given_spam
    ham_parameters[distinct_word] = prob_word_given_ham

## Classifying a New Message

Now that we have all the needed constants and parameters we can progress to creating the spam filter. This function:

- Takes in a new message as input $ (w_1, w_2,..., w_n) $
- Calculates $ P(Spam | (w_1, w_2,..., w_n) $ and $ P(Ham | (w_1, w_2,..., w_n) $
- Compares both values, and:
    - If $ P(Spam | (w_1, w_2,..., w_n) > P(Ham | (w_1, w_2,..., w_n)$ then the message is classified as spam
    - If $ P(Spam | (w_1, w_2,..., w_n) < P(Ham | (w_1, w_2,..., w_n)$ then the message is classified as non-spam
    - If $ P(Spam | (w_1, w_2,..., w_n) = P(Ham | (w_1, w_2,..., w_n)$ then the algorithm may ask for human help

We will use the functions for $ P(Spam | (w_1, w_2,..., w_n) $ and $ P(Ham | (w_1, w_2,..., w_n) $ that we displayed in the last section to develop our function to classify messages below:

In [16]:
def classify(message):
    
    # format new message to match current standards
    message = re.sub('\W', ' ', message)
    message = message.lower().split()
    
    # prob of message being spam is a product of the prob of a message being spam times the prob of each word being found in spam messages
    # spam/ham base probabilities will be used as a base value in case none of the words in the vocabulary are found in the message
    p_spam_given_message = p_spam
    p_ham_given_message = p_ham
    
    for word in message:
        if word in spam_parameters:
            p_spam_given_message *= spam_parameters[word]
        if word in ham_parameters:
            p_ham_given_message *= ham_parameters[word]        

    if p_ham_given_message > p_spam_given_message:
        return 'ham'
    elif p_ham_given_message < p_spam_given_message:
        return 'spam'
    else:
        return 'Human requested'      

We can test an obviously spam message and one that sounds normal below:

In [17]:
classify('WINNER!! This is the secret code to unlock the money: C3421.')

'spam'

In [18]:
classify("Sounds good, Tom, then see u there")

'ham'

## Measuring the Spam Filter's Accuracy

Now that our function seems to be working, we will create a new column in our testing dataset with the results of our algorithm's prediction. If we compare this preduction to the real values (recall that all of our sample messages were already classified as spam/non-spam by humans before), then we can determine how accurate said algorithm is for our needs.

In [22]:
# create new column in testing dataset using the classify() function
test['prediction'] = test['SMS'].apply(classify)

In [23]:
# check if any message needs human intervention
human_needed_count = len(test[test['prediction'] == 'Human requested'])
print('The number of messages needing human intervention is ', human_needed_count)

The number of messages needing human intervention is  1


Perhaps predictably due to the number of decimals involved, only one message has been marked as needing a human. Now we can test the accuracy of our algorithm by comparing how often our prediction was the same to the existing label:

In [26]:
# bool mask for correct results
correct_prediction_mask = test['Label'] == test['prediction']

# number of correct guesses
correct_prediction_rows = len(test[correct_prediction_mask])

# number of total test messages
total_messages = len(test)

# % of correct guesses

accuracy_percent = round((correct_prediction_rows / total_messages) *100, 2)

print('Out of {} messages, our algorithm guesses {} right. This means our algorithm filtered out spam with {}% accuracy!'.format(total_messages, correct_prediction_rows, accuracy_percent))

Out of 1114 messages, our algorithm guesses 1100 right. This means our algorithm filtered out spam with 98.74% accuracy!


## Next Steps

In this exercise we built a spam filter for SMS messages using the Multinomial Naive Bayes algorithm. Our initial objective was to obtain at least 80% of accuracy, but we managed to do a lot better than that at 98.74% accuracy.

Some of the potential next steps we could take from here are:

- Checking what went wrong with the 14 messages that were classifies incorrectly
- Making the filtering more complex by making the algorithm case sensitive